In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from IPython.display import display

from scipy.optimize import curve_fit
from scipy.signal import find_peaks

import copy

In [12]:
class Frontend():
    
    def __init__(self, backend):
        
        self.backend = backend

        
        # Generate the figure output widget
        # Plot the initial data and configure the display settings
        self.output = widgets.Output(label=None)
        
        with self.output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
            
        self.line, = self.ax.plot(self.backend.x, self.backend.y, 'k')
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)
        
        # list of all plt.line objects for future generated fits
        self.fit_curve_lines = []
        # Line object for the current fit line
        self.current_fit_line, = self.ax.plot([])
        
        
        # High-level control box
        self.ctrl_b1 = widgets.Button(description='Fit data in range')
        self.ctrl_b2 = widgets.Button(description='Cancel', disabled=True)

        self.ctrl_box = widgets.VBox(
            [
                #widgets.HTML(
                #                value='<b>Control</b>', 
                #                description=''
                #            ),
                widgets.Label('Control box'),
                self.ctrl_b1,
                self.ctrl_b2
            ]
        )
        
        
        # Control box widget observe and update
        self.ctrl_b1.on_click(self.__goto_FitMode)
        self.ctrl_b2.on_click(self.__goto_SearchMode)
        
        
        
        # Fit control box
        self.backend.cur_fitmodel = self.backend.fit_models[0]
        
        self.fitmodel_dropdown = widgets.Dropdown(value=self.backend.model_names[0], 
                                             options=self.backend.model_names, 
                                             description='Fit Model',
                                             disabled=True
                                            )
        # Watch the dropdown
        self.fitmodel_dropdown.observe(self.__init_FitModel, 'value')      
        
        
        # For each parameter, generate an input floattext widget
        # Create an observer and write the parameter values to the backend
        self.param_input_widgets = []
        self.backend.param_vect = []
        
        for k in range(self.backend.cur_fitmodel.num_params):
            
            # Create the new input text widgets
            self.param_input_widgets.append(
                widgets.FloatText(value=self.backend.cur_fitmodel.param_default[k], 
                                  description=self.backend.cur_fitmodel.param_names[k], 
                                  continuous_update=False,
                                  disabled=True
                                 )
            )

            # Construct the parameter backend values
            self.backend.param_vect.append(self.backend.cur_fitmodel.param_default[k])
            
            
            # Observe the input widgets
            self.param_input_widgets[k].observe(self.backend.update_param, 'value')

        
        # Make the box of parameter input widgets
        self.param_box = widgets.VBox(
            self.param_input_widgets 
        )

        # Join the dropdown and fit parameter widgets
        self.fit_box = widgets.VBox([self.fitmodel_dropdown, self.param_box])

        
        # Fit results widget box
        self.trigger_fit_button = widgets.Button(description='Run fit', disabled=True)
        
        # Attempt to fit when triggered
        self.trigger_fit_button.on_click(self.backend.fit_data)
        
        # Create a set of output text widgets to display the results
        self.param_output_widgets = []
        for k in range(self.backend.cur_fitmodel.num_params):
            
            self.param_output_widgets.append(
                widgets.FloatText(value=None, 
                                  description=self.backend.cur_fitmodel.param_names[k], 
                                  continuous_update=False,
                                  disabled=True
                                 )
            )
            
        
        
        
        # Button to accept the fit
        self.accept_fit_button = widgets.Button(description='Accept fit', disabled=True)
        
        # When triggered accept the fit and return to search mode
        self.accept_fit_button.on_click(self.__accept_fit)
        
        
        self.output_box = widgets.VBox(
            [self.trigger_fit_button, self.accept_fit_button] + self.param_output_widgets 
        )
        
        # Set the frontend
        box_layout = widgets.Layout(
                border='solid 1px black',
                margin='0px 10px 10px 0px',
                padding='5px 5px 5px 5px')
        
        self.ctrl_box.layout = box_layout
        self.fit_box.layout = box_layout
        self.output.layout = box_layout
        self.output_box.layout = box_layout
        
        self.gui = widgets.VBox(
            [
                self.ctrl_box,
                
                widgets.HBox(
                    [
                        widgets.VBox(
                            [
                                self.fit_box,
                                self.output_box
                            ]
                        ),
                        self.output
                    ]
                ) 
            ]
        )
        
        # Display the frontend
        display(self.gui)
        
        
    def __goto_FitMode(self, change):
        
        self.ctrl_b1.disabled=True
        self.ctrl_b2.disabled=False
        self.backend.state = 'Fit'
        
        self.fitmodel_dropdown.disabled = False
        for param in self.param_box.children:
            param.disabled = False
            
        # Enable the fit buttons
        self.trigger_fit_button.disabled = False
        #self.accept_fit_button.disabled = False
        
        # Get the data in the domain of the fit range
        self.backend.fit_x, self.backend.fit_y = self.backend.get_data_in_range(self.ax.get_xlim())

        # Update the plot
        self.line.set_alpha(0.1)
        self.fit_region_data_line, = self.ax.plot(self.backend.fit_x, self.backend.fit_y, 'ko')
        
        # Remove the accepted fit lines
        for line in self.fit_curve_lines:
            line.remove()
            
       
    def __goto_SearchMode(self, change):
        
        self.ctrl_b1.disabled=False
        self.ctrl_b2.disabled=True 
        self.backend.state = 'Search'
        
        self.fitmodel_dropdown.disabled = True
        for param in self.param_box.children:
            param.disabled = True
        
        # Disable the fit buttons
        self.trigger_fit_button.disabled = True
        self.accept_fit_button.disabled = True
        
        self.line.set_alpha(1)
        self.fit_region_data_line.remove()
        
        # Remove the current fit line
        self.current_fit_line.remove()
        
        # Add back the accepted fit lines
        for line in self.fit_curve_lines:
            self.ax.add_line(line)
        
        
    def __init_FitModel(self, change):
        
        # Get the correct model
        for model in self.backend.fit_models:
            if model.name == change.new:
                self.backend.cur_fitmodel = model
                break
                
        else:
            self.backend.cur_fitmodel = None
            
        if self.backend.cur_fitmodel is not None:

            # For each parameter, generate an input floattext widget
            # Create an observer and write the parameter values to the backend
            self.param_input_widgets = []
            self.param_output_widgets = []
            self.backend.param_vect = []

            for k in range(self.backend.cur_fitmodel.num_params):

                # Create the new input text widgets
                self.param_input_widgets.append(
                    widgets.FloatText(value=self.backend.cur_fitmodel.param_default[k], 
                                      description=self.backend.cur_fitmodel.param_names[k], 
                                      continuous_update=False,
                                      disabled=False
                                     )
                )
                
                # Create the new output text widgets
                self.param_output_widgets.append(
                widgets.FloatText(value=None, 
                                  description=self.backend.cur_fitmodel.param_names[k], 
                                  continuous_update=False,
                                  disabled=True
                                 )
                )

                # Construct the parameter backend values
                self.backend.param_vect.append(self.backend.cur_fitmodel.param_default[k])

                # Observe the input widgets
                self.param_input_widgets[k].observe(self.backend.update_param, 'value')

        
        # Update the widget container to include updated list
        self.param_box.children = self.param_input_widgets
        self.output_box.children = [self.trigger_fit_button, self.accept_fit_button] + self.param_output_widgets
            
        
    def update_results(self):
        
        # Update the output results widgets
        for output_widget, output in zip(self.param_output_widgets, self.backend.fit_params):
            output_widget.value = output
            
        # Plot the fit line
        self.current_fit_line.remove()
        self.current_fit_line, = self.ax.plot(self.backend.fit_x, self.backend.fit_result, 'C0', linewidth=2)
        
        # Enable the accept fit button
        self.accept_fit_button.disabled = False
        
    def __accept_fit(self, change):
        
        print('attempting to save with class')
        print(self.backend.fit_class)
        
        # Append the fitclass
        self.backend.fits.append(
            self.backend.fit_class(
                x = self.backend.fit_x, 
                y = self.backend.fit_y, 
                fit_model = self.backend.cur_fitmodel,
                fit_y = self.backend.fit_result,  
                fit_params = self.backend.fit_params, 
                fit_covmat = self.backend.fit_covmat,
                meta=None
            )
        )
        
        self.fit_curve_lines.append(
            self.current_fit_line
        )
        
        self.ax.add_line(self.current_fit_line)
        
        self.__goto_SearchMode(change)

In [13]:
class Backend():
    
    def __init__(self,
                 fit_models,       # List of fit models
                 data_x,           # Data x vector
                 data_y,           # Data y vector
                 fit_class,        # Class of fit result
                 data_meta = None  # Dictionary of data metadata
                ):
        
        self.fit_models = fit_models
        self.model_names = [model.name for model in self.fit_models]
        
        self.x = data_x
        self.y = data_y
        self.meta = data_meta
        
        self.fit_class = fit_class
        
        self.fits = []  # Set of fits generated by the backend
        
        self.state = 'Search'
        
        # Data to fit
        self.fit_x = None
        self.fit_y = None
        
        # The current fit model being used
        self.cur_fitmodel = None
        
        # Guess for parameters to start fit
        self.param_vect = None
        
        # Output parameters and covariance matrix from fit attempt
        self.fit_params = None
        self.fit_covmat = None    
        
        # y data for the most recent fit result
        self.fit_result = None
        
        # Lastly generate the frontend
        self.frontend = Frontend(self)
        

        
        
        
    def get_data_in_range(self, limits):
        mask = np.argwhere(np.logical_and(self.x > limits[0], self.x < limits[1])).ravel()
    
        return self.x[mask], self.y[mask]
    
    
    
    
    def update_param(self, change):
        
        index = self.frontend.param_input_widgets.index(change.owner)
        
        print('UPDATING: ' + str(index))

        self.param_vect[index] = change.new
        
        print('New param vect: ' + str(self.param_vect))
        
        # Probably trigger the system to fit
        # Maybe want to add a toggle for auto fitting or not
        
        
        
        
    def fit_data(self, change):
        
        print('Attempting to fit with:')
        print('\tFitModel = ' + self.cur_fitmodel.name)
        
        #try:
            
        # Attempt a curve fit
        self.fit_params, self.fit_covmat = curve_fit(

              self.cur_fitmodel.f,                # Fit model function (callable)
              self.fit_x,                         # x data to fit
              self.fit_y,                         # y data to fit
              p0=self.param_vect,                 # Initial guess for fit params
              bounds=(                            # Parameter bounds
                  self.cur_fitmodel.param_min,        # Parameter lower bound
                  self.cur_fitmodel.param_max         # Parameter upper bound
              )

        )


        # Compute the updated fit line
        self.fit_result = self.cur_fitmodel.f(self.fit_x, *self.fit_params)
        
        # Update the front end
        self.frontend.update_results()
        
        #except Exception as e:
            
         #   print('Error encountered with fit:\n\t' + str(e))
    

In [14]:
class Fit(object):

    def __init__(self,  
                 x, 
                 y, 
                 fit_model,
                 fit_y, 
                 fit_params, 
                 fit_covmat,
                 meta=None
                ):

        self.x = x
        self.y = y
        self.fit_model = fit_model
        self.fit_y = fit_y
        self.fit_params = fit_params
        self.fit_covmat = fit_covmat
        
        self.meta = meta

class ResonanceFit1(Fit):
    
    '''
    Fit object for use with single-peak Lorentzian
    '''

    def __init__(self,  
                 x, 
                 y, 
                 fit_model,
                 fit_y,  
                 fit_params, 
                 fit_covmat,
                 meta=None
                ):
        
        super().__init__(x, 
                         y, 
                         fit_model,
                         fit_y, 
                         fit_params, 
                         fit_covmat,
                         meta
                        )
        
        self.Q = fit_params[1] / fit_params[2]
        
    # Functions to compare the Q factors directly
    def __eq__(self, other):
        return self.Q == other.Q

    def __lt__(self, other):
        return self.Q < other.Q

In [15]:
class FitModel():
    
    def __init__(self,
                 name,                 # Name
                 param_names,          # name of parameters
                 param_default,        # Default values of parameters
                 param_min,            # Parameter minimum possible
                 param_max,            # Parameter maximum possible
                 param_dict = None     # Parameter dictionary ``{'param': '[description]'}``
                ):
        
        self.name = name
        self.param_names = param_names
        self.param_default = param_default
        self.param_min = param_min
        self.param_max = param_max
    
        self.num_params = len(param_names)
    
    
    
    def f(self, x, *params):
        pass


# Test models
class Model_1(FitModel):
    
    def __init__(self):
        super().__init__(
            name = 'Model_1',
            param_names = ['a'],
            param_default = [0],
            param_min = [-1],
            param_max = [1]
        )
        
class Model_2(FitModel):
    
    def __init__(self):
        super().__init__(
            name = 'Model_2',
            param_names = ['a', 'b'],
            param_default = [0, 0],
            param_min = [-1, -1],
            param_max = [1, 1]
        )
        
class Model_3(FitModel):
    
    def __init__(self):
        super().__init__(
            name = 'Model_3',
            param_names = ['a', 'b', 'c'],
            param_default = [0, 0, 0],
            param_min = [-1, -1, -1],
            param_max = [1, 1, 1]
        )
        
        
class Lorentzian_p1(FitModel):
    
    def __init__(self):
        super().__init__(
            name = 'Lorentzian_poly1',
            param_names = ['a', 'mu', 'gamma', 'p0', 'p1'],
            param_default = [1, 0, 1, 0, 0],
            param_min = [-np.inf, 0, 0, -np.inf, -np.inf],
            param_max = [np.inf, np.inf, np.inf, np.inf, np.inf],
            param_dict = {
                            'a':     'peak amplitude',
                            'mu':    'peak center' ,
                            'gamma': 'peak width (FWHM)',
                            'p0':    '0th order poly coeff (constant)',
                            'p1':    '1st order poly coeff (linear)'
                         }
        )
        
    def f(self, x, *params):
        a, mu, gamma, p0, p1 = params
        
        lorentzian = a * (gamma**2/4) / ( (x - mu)**2 + gamma**2/4 )
        background = p0 + p1 * (x - mu)
        
        return lorentzian + background
    

class Lorentzian_p2(FitModel):
    
    def __init__(self):
        super().__init__(
            name = 'Lorentzian_poly2',
            param_names = ['a', 'mu', 'gamma', 'p0', 'p1', 'p2'],
            param_default = [1, 0, 1, 0, 0, 0],
            param_min = [-np.inf, 0, 0, -np.inf, -np.inf, -np.inf],
            param_max = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
            param_dict = {
                            'a':     'peak amplitude',
                            'mu':    'peak center' ,
                            'gamma': 'peak width (FWHM)',
                            'p0':    '0th order poly coeff (constant)',
                            'p1':    '1st order poly coeff (linear)',
                            'p2':    '2nd order poly coeff (quadratic)'
                         }
        )
        
    def f(self, x, *params):
        a, mu, gamma, p0, p1, p2 = params
        
        lorentzian = a * (gamma**2/4) / ( (x - mu)**2 + gamma**2/4 )
        background = p0 + p1 * (x - mu) + p2 * (x - mu)**2
        
        return lorentzian + background
        

In [16]:
lorentzian_p1 = Lorentzian_p1()

x = np.linspace(0,1,250)
y = np.random.normal(scale=0.05, size=(250)) 
y = y + lorentzian_p1.f(x, *[0.6, 0.3, 0.04, 0.1, 0.04]) 
y = y + lorentzian_p1.f(x, *[0.4, 0.7, 0.03, 0, 0])

#fig, ax = plt.subplots()
#plt.plot(x,y)
#plt.show()

In [22]:
b = Backend([Lorentzian_p1(), Lorentzian_p2()],x,y,ResonanceFit1)

Attempting to fit with:
	FitModel = Lorentzian_poly1
UPDATING: 1
New param vect: [1, 0.3, 1, 0, 0]
Attempting to fit with:
	FitModel = Lorentzian_poly1
attempting to save with class
<class '__main__.ResonanceFit1'>
UPDATING: 1
New param vect: [1, 0.7, 1, 0, 0]
Attempting to fit with:
	FitModel = Lorentzian_poly1
attempting to save with class
<class '__main__.ResonanceFit1'>


In [23]:
for fit in b.fits:
    print(fit.Q)

7.345012072489262
22.415527272715863
